# Re-rank chunks and generate answers

This code demonstrate how to evaluate chunks in parallel generating the percentage of similarity with the question and the answer (part of text in the chunk more appropiate to answer the question).

The output is the chunks more relevant to answer the question and the answer generated with those chunks.

## Prerequisites

+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
+ An Azure OpenAI service with the service name and an API key.
+ A deployment of the text-embedding-ada-002 embedding model on the Azure OpenAI Service.
+ An Azure AI Search service with the end-point, API Key and the index name to create.

We used Python 3.12.5, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install openai
! pip install azure-search-documents

## Import packages and create AOAI client

In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import sys
sys.path.append('../..')
from rag_utils import calculate_rank, semantic_hybrid_search_with_filter, get_filtered_chunks, generate_answer

# Load environment variables from .env
load_dotenv(override=True)

# AZURE AI SEARCH
ai_search_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
ai_search_apikey = os.environ["SEARCH_SERVICE_QUERY_KEY"]
ai_search_index_name = os.environ["SEARCH_INDEX_NAME"]
ai_search_credential = AzureKeyCredential(ai_search_apikey)
# Create Azure AI Search client
ai_search_client = SearchClient(endpoint=ai_search_endpoint, index_name=ai_search_index_name, credential=ai_search_credential)

aoai_api_version = '2024-02-15-preview'

# AOAI FOR ANSWER GENERATION
aoai_answer_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_answer_apikey = os.environ["AZURE_OPENAI_API_KEY"]
aoai_answer_model_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
# Create AOAI client for answer generation
aoai_answer_client = AzureOpenAI(
    azure_deployment=aoai_answer_model_name,
    api_version=aoai_api_version,
    azure_endpoint=aoai_answer_endpoint,
    api_key=aoai_answer_apikey
)

# AZURE OPENAI FOR RERANKING
aoai_rerank_endpoint = os.environ["AZURE_OPENAI_RERANK_ENDPOINT"]
azure_openai_rerank_key = os.environ["AZURE_OPENAI_RERANK_API_KEY"]
rerank_model_name = os.environ["AZURE_OPENAI_RERANK_DEPLOYMENT_NAME"]
# Create AOAI client for reranking
aoai_rerank_client = AzureOpenAI(
    azure_deployment=rerank_model_name,
    api_version=aoai_api_version,
    azure_endpoint=aoai_rerank_endpoint,
    api_key=azure_openai_rerank_key
)

# AZURE OPENAI FOR EMBEDDING
aoai_embedding_endpoint = os.environ["AZURE_OPENAI_EMBEDDING_ENDPOINT"]
azure_openai_embedding_key = os.environ["AZURE_OPENAI_EMBEDDING_API_KEY"]
embedding_model_name = os.environ["AZURE_OPENAI_EMBEDDING_NAME_ADA"]
# Create AOAI client for embedding creation (ADA)
aoai_embedding_client = AzureOpenAI(
    azure_deployment=embedding_model_name,
    api_version=aoai_api_version,
    azure_endpoint=aoai_embedding_endpoint,
    api_key=azure_openai_embedding_key
)

# CONSTANTS
MAX_DOCS = 20 # Maximum number of documents to retrieve in the query
EMBEDDING_FIELDS = "embeddingTitle, embeddingContent" # Vector fields to search for
SELECT_FIELDS=["id", "title", "content"] # Fields to retrieve in the search

In [2]:
import pandas as pd

def show_results(results, query, rerank=False):
    #print(f'query: {query}, num results: {results.get_count()}')
    data = []
    for i, result in enumerate(results):
        if rerank:
            confidence, answer = calculate_rank(aoai_rerank_client, rerank_model_name, result['title'] + ". " + result['content'], query)
            response = f'confidence: {confidence}, answer: {answer}'
        else:
            response = 'n/a'

        data.append(
            [result["id"],
             result["title"],
             result["content"], 
             result["@search.score"],
             response
            ]
        )
        if i + 1 == MAX_DOCS: break # Stops at the maximum number of documents

    return pd.DataFrame(data, columns=["id", "title", "content", "@search.score", "rerank"])

In [7]:
query = "How can I optimize the network at home?"
results = semantic_hybrid_search_with_filter(ai_search_client, query, aoai_embedding_client, embedding_model_name, EMBEDDING_FIELDS, MAX_DOCS, SELECT_FIELDS)
show_results(results, query, True)

,id,title,content,@search.score,rerank
0,145,Providing tips for home network optimization.,applications during peak times.\n\nSchedule D...,0.049727,"confidence: 80, answer: ""To optimize the netwo..."
1,133,Providing guidance on streaming bandwidth needs.,add up quickly. It is important to consider t...,0.038384,"confidence: 90, answer: ""To optimize the netwo..."
2,172,Understanding internet usage patterns.,plan.\n\nCommon Activities: Inquire about the...,0.013699,"confidence: 80, answer: ""To optimize the netwo..."
3,116,Improving Wi-Fi signal through placement.,repositioning the router to a more central lo...,0.036664,"confidence: 80, answer: ""Improving Wi-Fi signa..."
4,118,Managing multiple devices connected to Wi-Fi.,"new Wi-Fi network within the home, linked to ...",0.035705,"confidence: 70, answer: ""To optimize the netwo..."
5,15,Assisting with IoT device connections.,features on the router (like firewall or MAC ...,0.024326,"confidence: 70, answer: ""To optimize the netwo..."
6,130,Providing guidance on energy-efficient network...,or sleep modes during times when the network ...,0.040190,"confidence: 80, answer: ""To optimize the netwo..."
7,47,Explaining network latency impacts.,of wireless connections to reduce latency.\n\...,0.015625,"confidence: 80, answer: ""To optimize the netwo..."
8,5,Assisting with bandwidth allocation for differ...,offer these advanced tips:\n\nThird-Party Fir...,0.047418,"confidence: 80, answer: ""To optimize the netwo..."
9,143,Providing tips for DIY network repairs.,the customer to note down any custom settings...,0.041445,"confidence: 80, answer: ""For optimizing the ne..."


## Calculate the re-ranking and generate the answer using the chunks as the context

In [3]:
query = "How can I optimize the network at home?"
results = semantic_hybrid_search_with_filter(ai_search_client, query, aoai_embedding_client, embedding_model_name, EMBEDDING_FIELDS, MAX_DOCS, SELECT_FIELDS)
chunks = get_filtered_chunks(aoai_rerank_client, rerank_model_name, results, query, MAX_DOCS)
if len(chunks) > 0:
    # Generate answer using the complete chunks
    answer = generate_answer(aoai_answer_client, aoai_answer_model_name, chunks, query, 'content')
else:
    answer = 'There it not content to generate the answer'
print(f'\nANSWER: {answer}')


ANSWER: ((142)) To optimize the network at home, follow these steps: 

- ((142)) Place the router in a central location, elevate it off the ground, and ensure it is free from obstructions. 
- ((142)) Change the Wi-Fi channel on the router to minimize interference from other devices. 
- ((142)) Regularly update the router firmware to ensure optimal performance. 
- ((142)) Consider switching to a dual-band router for better signal strength. 

((142)) These actions can help improve connectivity and reduce issues like interference and overlapping channels. ((142))


## Calculate the re-ranking and generate the answer using the answer generated by the re-ranker as the context

In [4]:
query = "How can I optimize the network at home?"
results = semantic_hybrid_search_with_filter(ai_search_client, query, aoai_embedding_client, embedding_model_name, EMBEDDING_FIELDS, MAX_DOCS, SELECT_FIELDS)
chunks = get_filtered_chunks(aoai_rerank_client, rerank_model_name, results, query, MAX_DOCS)
if len(chunks) > 0:
    # Generate answer using the 'answer' generated by the re-ranker
    answer = generate_answer(aoai_answer_client, aoai_answer_model_name, chunks, query, 'answer')
else:
    answer = 'There it not content to generate the answer'
print(f'\nANSWER: {answer}')


ANSWER: ((142)) To optimize the network at home, follow these steps: 

- Place the router centrally in the home for optimal coverage. ((142))
- Minimize physical obstructions between the router and devices. ((142))
- Update the router's firmware to the latest version. ((142))
- Consider upgrading to a newer router with better bandwidth management features. ((142))
- Schedule heavy downloads or uploads during off-peak hours. ((142))
- Limit the number of devices connected to the network simultaneously. ((142))
- Ensure devices using the same frequency are not causing interference. ((142))
- Switch to the 5GHz frequency band if the router supports it. ((142))
- Consider discussing an upgrade to a higher-tier internet plan. ((142))

To verify actual bandwidth, perform a speed test on a reliable website, ensuring no other devices are using the internet during the test, and compare the results to the subscribed plan. ((142))
